# AGYW_PREV DATIM

In [ ]:
# load the modules
#%run layeringOfServices.py
from layeringOfServices import dreams_mastersheet
#multiples outputs
from IPython.core.interactiveshell import InteractiveShell
InteractiveShell.ast_node_interactivity = "all"

In [ ]:
dreams_mastersheet.dtypes


## Datim Pour les 3 definitions

----

1. Number of individual AGYW that have fully completed the entire DREAMS primary package of services but have not received any services beyond the primary package.

2. Number of individual AGYW that have fully completed the entire DREAMS primary package of services AND at least one additional secondary service.

3. Number of individual AGYW that have fully completed at least one DREAMS primary package of services but not the full primary package of services.




----

## Task it 

In [ ]:
# Nbre d'observation
dreams_mastersheet.code.count()

In [ ]:
# les tranches d'age
dreams_1014 = dreams_mastersheet[dreams_mastersheet.age_range == "10-14"]
dreams_1519 = dreams_mastersheet[dreams_mastersheet.age_range == "15-19"]
dreams_2024 = dreams_mastersheet[dreams_mastersheet.age_range == "20-24"]

In [ ]:
dreams_1014.code.count()
dreams_1519.code.count()
dreams_2024.code.count()

In [ ]:
dreams_1014 = dreams_1014[dreams_1014.timeOn_system == "required_Time_on"]
dreams_1519 = dreams_1519[dreams_1519.timeOn_system == "required_Time_on"]
dreams_2024 = dreams_2024[dreams_2024.timeOn_system == "required_Time_on"]

In [ ]:
dreams_1014.code.count()
dreams_1519.code.count()
dreams_2024.code.count()

In [ ]:
dreams_valid = dreams_mastersheet[dreams_mastersheet.timeOn_system == "required_Time_on"]

dreams_valid = dreams_valid[(dreams_valid.age_range !="not_valid_age") & (dreams_valid.age_range != "25-29")]

dreams_valid.code.count()

## Process Primary Only
--- 
Number of individual AGYW that have fully completed the entire DREAMS primary package of services but have not received any services beyond the primary package.

In [ ]:
# primary services function

def primFunc(df):
    if (df.ps_10_14 == "curriculum-servis" and df.hts == "errata" and df.condoms == "errata" and df.post_care_treatment == "no" and df.socio_eco_app == "no" ):
        return "full_primary_only"
    elif (df.ps_15_19 == "condoms&curriculum" and df.hts == "errata"  and df.post_care_treatment == "no" and df.socio_eco_app == "no" ):
        return "full_primary_only"
    elif (df.ps_20_24 == "condoms&hts&curriculum" and df.post_care_treatment == "no" and df.socio_eco_app == "no" ):
        return "full_primary_only"
    else:
        "invalid"

In [ ]:
dreams_1014["primary_only"] = dreams_1014.apply(lambda df: primFunc(df),axis=1)
dreams_1519["primary_only"] = dreams_1519.apply(lambda df: primFunc(df),axis=1)
dreams_2024["primary_only"] = dreams_2024.apply(lambda df: primFunc(df),axis=1)

dreams_valid["primary_only"] = dreams_valid.apply(lambda df: primFunc(df),axis=1)

In [ ]:
dreams_valid = dreams_valid[dreams_valid.primary_only == "full_primary_only"].code.count()
dreams_valid

In [ ]:
dreams_1014 = dreams_1014[dreams_1014.primary_only == "full_primary_only"].code.count()
dreams_1014

In [ ]:
dreams_1519 = dreams_1519[dreams_1519.primary_only == "full_primary_only"].code.count()
dreams_1519

In [ ]:
dreams_2024 = dreams_2024[dreams_2024.primary_only == "full_primary_only"].code.count()
dreams_2024

In [ ]:
dreams_valid_pivot = dreams_valid.pivot_table(index=["age_range"],columns=["agyw_period_range"], values="code",aggfunc="count",fill_value=0,margins=True,margins_name="Total")
dreams_valid_pivot.rename_axis(index={"age_range":"Age"},columns={"agyw_period_range":"Time"},inplace=True)
dreams_valid_pivot_final = dreams_valid_pivot[:-1]
dreams_valid_pivot_final

In [ ]:
dreams_1014_pivot = dreams_1014.pivot_table(index=["age_range"],columns=["agyw_period_range"], values="code",aggfunc="count",fill_value=0,margins=True,margins_name="Total")
dreams_1014_pivot.rename_axis(index={"age_range":"Age"},columns={"agyw_period_range":"Time"},inplace=True)
dreams_1014_pivot_final = dreams_1014_pivot[:-1]
dreams_1014_pivot_final

In [ ]:
dreams_1519_pivot = dreams_1519.pivot_table(index=["age_range"],columns=["agyw_period_range"], values="code",aggfunc="count",fill_value=0,margins=True,margins_name="Total")
dreams_1519_pivot.rename_axis(index={"age_range":"Age"},columns={"agyw_period_range":"Time"},inplace=True)
dreams_1519_pivot_final = dreams_1519_pivot[:-1]
dreams_1519_pivot_final

In [ ]:
dreams_2024_pivot = dreams_2024.pivot_table(index=["age_range"],columns=["agyw_period_range"], values="code",aggfunc="count",fill_value=0,margins=True,margins_name="Total")
dreams_2024_pivot.rename_axis(index={"age_range":"Age"},columns={"agyw_period_range":"Time"},inplace=True)
dreams_2024_pivot_final = dreams_2024_pivot[:-1]
dreams_2024_pivot_final